In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import preprocessing

In [2]:
train = pd.read_csv('/kaggle/input/predict-the-positions-and-speeds-of-600-satellites/jan_train.csv')
test = pd.read_csv('/kaggle/input/predict-the-positions-and-speeds-of-600-satellites/jan_test.csv')
key = pd.read_csv('/kaggle/input/predict-the-positions-and-speeds-of-600-satellites/answer_key.csv')

In [3]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 503227 entries, 0 to 503226
Data columns (total 15 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   id      503227 non-null  int64  
 1   epoch   503227 non-null  object 
 2   sat_id  503227 non-null  int64  
 3   x       503227 non-null  float64
 4   y       503227 non-null  float64
 5   z       503227 non-null  float64
 6   Vx      503227 non-null  float64
 7   Vy      503227 non-null  float64
 8   Vz      503227 non-null  float64
 9   x_sim   503227 non-null  float64
 10  y_sim   503227 non-null  float64
 11  z_sim   503227 non-null  float64
 12  Vx_sim  503227 non-null  float64
 13  Vy_sim  503227 non-null  float64
 14  Vz_sim  503227 non-null  float64
dtypes: float64(12), int64(2), object(1)
memory usage: 57.6+ MB


In [4]:
train['epoch'] = pd.to_datetime(train['epoch'])

In [5]:
X = train[['id', 'sat_id', 'x_sim','y_sim', 'z_sim', 'Vx_sim', 'Vy_sim', 'Vz_sim']].copy()
Y = train[['x', 'y', 'z', 'Vx', 'Vy', 'Vz']]
from sklearn import preprocessing
min_max_scaler = preprocessing.MinMaxScaler()
X_scale = min_max_scaler.fit_transform(X)

In [6]:
from sklearn import preprocessing as pp
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR
from sklearn.metrics import accuracy_score, confusion_matrix
import random
import pickle
from sklearn.utils import shuffle
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.metrics import *
from sklearn import *

In [7]:
from sklearn.model_selection import train_test_split

X_train, X_val_and_test, Y_train, Y_val_and_test = train_test_split(X_scale, Y, test_size=0.1)
X_val, X_test, Y_val, Y_test = train_test_split(X_val_and_test, Y_val_and_test, test_size=0.5)

print(X_train.shape, X_val.shape, X_test.shape, Y_train.shape, Y_val.shape, Y_test.shape)

(452904, 8) (25161, 8) (25162, 8) (452904, 6) (25161, 6) (25162, 6)


In [8]:
test = pd.read_csv('/kaggle/input/predict-the-positions-and-speeds-of-600-satellites/jan_test.csv')
test = test[['id', 'sat_id', 'x_sim','y_sim', 'z_sim', 'Vx_sim', 'Vy_sim', 'Vz_sim']].copy()
test_scale = min_max_scaler.transform(test)

# MULTI-LAYER PERCEPTRON MODEL

In [10]:
# LINEAR CLASSIFICATION AT OUTPUT LAYER - EPOCHS = 5 
from keras.models import Sequential
from keras.layers import Dense

# define the model
model = Sequential()
model.add(Dense(64, input_dim=8, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(6, activation='linear'))

# compile the model
model.compile(loss='mean_squared_error', optimizer='adam', metrics = ['accuracy'])

# fit the model to the training data
model.fit(X_train, Y_train, epochs=5, batch_size=32, validation_data=(X_val, Y_val))


Epoch 1/5
14154/14154 [==============================] - 44s 3ms/step - loss: 243347104.0000 - accuracy: 0.4417 - val_loss: 198437344.0000 - val_accuracy: 0.5199
Epoch 2/5
14154/14154 [==============================] - 43s 3ms/step - loss: 186180896.0000 - accuracy: 0.5673 - val_loss: 163571456.0000 - val_accuracy: 0.6328
Epoch 3/5
14154/14154 [==============================] - 43s 3ms/step - loss: 123156168.0000 - accuracy: 0.7139 - val_loss: 93304200.0000 - val_accuracy: 0.7571
Epoch 4/5
14154/14154 [==============================] - 43s 3ms/step - loss: 91372224.0000 - accuracy: 0.7471 - val_loss: 87281536.0000 - val_accuracy: 0.7420
Epoch 5/5
14154/14154 [==============================] - 42s 3ms/step - loss: 87668608.0000 - accuracy: 0.7344 - val_loss: 84173192.0000 - val_accuracy: 0.7345


In [13]:
# SIGMOID CLASSIFICATION - 1 EPOCH 

from keras.models import Sequential
from keras.layers import Dense

# define the model
model = Sequential()
model.add(Dense(64, input_dim=8, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(6, activation='sigmoid'))

# compile the model
model.compile(loss='mean_squared_error', optimizer='adam', metrics = ['accuracy'])

# fit the model to the training data
model.fit(X_train, Y_train, epochs=1, batch_size=32, validation_data=(X_val, Y_val))


14154/14154 [==============================] - 40s 3ms/step - loss: 287436672.0000 - accuracy: 0.7671 - val_loss: 285982048.0000 - val_accuracy: 0.8138


In [14]:
model.save('MLPmodel.h5')

# EXTENDED GRADIENT BOOST MODEL 

In [27]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)

# Convert the data into DMatrix format, which is the format expected by XGBoost
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

# Set the parameters for the XGBoost model
param = {
    'max_depth': 5, 
    'eta': 0.1, 
    'objective': 'reg:squarederror'
}

# Train the model using XGBoost
model = xgb.train(param, dtrain, num_boost_round=100)

# Make predictions on the test data
y_pred = model.predict(dtest)

# Evaluate the model using mean squared error
mse = mean_squared_error(y_test, y_pred)
print("Mean squared error:", mse)

# Use the trained model to make a prediction on SAMPLE DATA
new_data = pd.DataFrame({'id': [1001], 'sat_id': [56], 'x_sim': [-1356.78], 'y_sim': [2356.78], 
                        'z_sim': [3456.78], 'Vx_sim': [56.78], 'Vy_sim': [-89.78], 'Vz_sim': [90.78]})
dnew = xgb.DMatrix(new_data) 
prediction = model.predict(dnew)

# The prediction will be an array with 6 values, representing the position and speed of the satellite
# [x, y, z, Vx, Vy, Vz]
print("Predicted position and speed of satellite:", prediction)


Mean squared error: 10906422.928717488
Predicted position and speed of satellite: [[-3.6037068e+03 -2.8779192e+03  1.0924578e+04  6.1423826e-01
  -1.7504978e+00  6.0496998e+00]]


In [ ]:
import joblib
test = pd.read_csv('/kaggle/input/predict-the-positions-and-speeds-of-600-satellites/jan_test.csv')
test = test[['id', 'sat_id', 'x_sim','y_sim', 'z_sim', 'Vx_sim', 'Vy_sim', 'Vz_sim']].copy()
# test_scale = min_max_scaler.transform(test)
test_data = pd.DataFrame(test, columns=['id', 'sat_id', 'x_sim','y_sim', 'z_sim', 'Vx_sim', 'Vy_sim', 'Vz_sim'])
dnew = xgb.DMatrix(test_data) 
model = joblib.load("/kaggle/input/sgb-model/xgb.joblib")
predictions = model.predict( dnew ) 
predictions_df = pd.DataFrame( predictions , columns = ['x', 'y', 'z', 'Vx', 'Vy', 'Vz'])
predictions_df.to_csv('predictions.csv',index = False ) 

In [19]:
import joblib 
import xgboost as xgb 
import pandas as pd 

model = joblib.load("/kaggle/input/sgb-model/xgb.joblib")
# convert test data into DMatrix object
test = pd.read_csv('/kaggle/input/predict-the-positions-and-speeds-of-600-satellites/jan_test.csv')
test = test[['id', 'sat_id', 'x_sim','y_sim', 'z_sim', 'Vx_sim', 'Vy_sim', 'Vz_sim']].copy()
test_data = pd.DataFrame(test, columns=['id', 'sat_id', 'x_sim','y_sim', 'z_sim', 'Vx_sim', 'Vy_sim', 'Vz_sim'])
dtest = xgb.DMatrix(test_data) 

# make predictions using the model
predictions = model.predict(dtest, ntree_limit=model.best_ntree_limit)

# load the key file
key = pd.read_csv('/kaggle/input/predict-the-positions-and-speeds-of-600-satellites/answer_key.csv')

# calculate the score
score = metrics.r2_score(key, predictions)
print('XGBoost score:', score)


/opt/conda/lib/python3.7/site-packages/xgboost/core.py:94: UserWarning: ntree_limit is deprecated, use `iteration_range` or model slicing instead.
  UserWarning


XGBoost score: 0.7571616636425053


In [1]:
# PKL TO JSON CONVERSION FOR MERN STACK APPLICATION 
import xgboost as xgb
import joblib
import json

# Load the model
model = joblib.load("/kaggle/input/sgb-model/xgb.joblib")

# Convert the model to a dictionary
model_dict = model.__dict__

# Serialize the dictionary to a JSON string
model_json = json.dumps(model_dict)

# Save the JSON string to a file
with open("xgb.json", "w") as f:
    f.write(model_json)

TypeError: Object of type c_void_p is not JSON serializable

# RANDOM FOREST REGRESSOR 

In [ ]:
from sklearn.ensemble import RandomForestRegressor

# create regressor object
regressor = RandomForestRegressor(n_estimators = 100)

# fit the regressor with x and y data
regressor.fit(X_train, Y_train)

regressor.score(X_test, Y_test)

In [26]:
from IPython.display import FileLink
FileLink(r'rf_regressor.joblib')

/kaggle/working/rf_regressor.joblib

In [ ]:
# Import the necessary libraries
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor

# Load the trained model
filename = '/kaggle/working/regressor.pkl'
loaded_model = pickle.load(open(filename, 'rb'))

# Create a new synthetic input data point
new_data = pd.DataFrame({'id': [1001], 'sat_id': [56], 'x_sim': [-1356.78], 'y_sim': [2356.78], 
                        'z_sim': [3456.78], 'Vx_sim': [56.78], 'Vy_sim': [-89.78], 'Vz_sim': [90.78]})

# Scale the new input data
min_max_scaler = preprocessing.MinMaxScaler()
new_data_scale = min_max_scaler.fit_transform(new_data)

# Use the trained model to make a prediction
prediction = loaded_model.predict(new_data_scale)

# The prediction will be an array with 6 values, representing the position and speed of the satellite
# [x, y, z, Vx, Vy, Vz]
print("Predicted position and speed of satellite:", prediction)

